In [2]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [3]:
user_features_df = pd.read_csv('kokh_user_features_df.csv', index_col='user_id')
user_features_df

In [4]:
post_features_df = pd.read_csv('kokh_post_features_df.csv', index_col='post_id')
post_features_df

In [7]:
def load_features_to_db(df: pd.DataFrame, name: str):
    df.to_sql(name, con=engine, if_exists='replace')
    return 'Loading completed'

In [8]:
load_features_to_db(user_features_df, 'kokh_user_features_df')

'Loading completed'

In [9]:
load_features_to_db(post_features_df, 'kokh_post_features_df')

'Loading completed'

In [10]:
df = pd.read_sql('SELECT * FROM kokh_user_features_df', con=engine, index_col='user_id')
df

,gender,age,country,city,exp_group,os,source,count_actions,category_of_age,cluster_feature,month,day,second,weekday,is_weekend,part_of_day
user_id,,,,,,,,,,,,,,,,
200,1,34,Russia,Degtyarsk,3,Android,ads,401,adult,9,10,14,28.239295,2,0,Evening
201,0,37,Russia,Abakan,0,Android,ads,748,adult,14,11,13,29.790667,2,0,Morning
202,1,17,Russia,Smolensk,4,Android,ads,724,young,18,10,14,28.861687,2,0,Morning
203,0,18,Russia,Moscow,1,iOS,ads,382,young,5,10,14,28.010417,2,0,Evening
204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads,161,adult,12,11,22,27.741379,1,0,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168548,0,36,Russia,Kaliningrad,4,Android,organic,382,adult,5,10,14,28.010417,2,0,Evening
168549,0,18,Russia,Tula,2,Android,organic,274,young,41,10,16,27.578182,2,0,Afternoon
168550,1,41,Russia,Yekaterinburg,4,Android,organic,407,adult,9,10,14,28.239295,2,0,Evening


In [11]:
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

def load_features() -> pd.DataFrame:
    query = 'SELECT * FROM nikita_efremov_feature'
    return batch_load_sql(query)

In [12]:
df_1 = load_features()
df_1

,user_id,gender,age,country,city,exp_group,os,source,count_actions,category_of_age,...,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,part_of_day,topic
0,200,1,34,Russia,Degtyarsk,3,Android,ads,401,adult,...,-0.004831,-0.004395,-0.002749,-0.002768,-0.003838,-0.003496,-0.005378,-0.003291,Afternoon,movie
1,201,0,37,Russia,Abakan,0,Android,ads,748,adult,...,-0.003363,-0.002610,-0.002905,-0.002383,-0.002466,-0.001954,-0.003388,-0.001596,Afternoon,movie
2,202,1,17,Russia,Smolensk,4,Android,ads,724,young,...,-0.002846,-0.003887,-0.003503,-0.004282,-0.002547,-0.003108,-0.002814,-0.003816,Evening,movie
3,203,0,18,Russia,Moscow,1,iOS,ads,382,young,...,-0.004616,-0.006999,-0.004537,-0.003671,-0.004947,-0.006008,-0.003755,-0.004889,Afternoon,movie
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads,161,adult,...,-0.009359,-0.003777,-0.006624,-0.003969,-0.003782,-0.005109,-0.006665,-0.005466,Afternoon,movie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic,382,adult,...,-0.003785,-0.002997,-0.003835,-0.002908,-0.003193,-0.003415,-0.004304,-0.003666,Evening,movie
163201,168549,0,18,Russia,Tula,2,Android,organic,274,young,...,-0.003896,-0.006984,-0.004236,-0.006059,-0.005308,-0.006353,-0.003941,-0.004360,Afternoon,movie
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic,407,adult,...,-0.002848,-0.001641,-0.001649,-0.004402,-0.000972,-0.001942,-0.003873,-0.001487,Evening,movie
163203,168551,0,38,Russia,Moscow,3,iOS,organic,525,adult,...,-0.005109,-0.002331,-0.002817,-0.003519,-0.003280,-0.004275,-0.002587,-0.004157,Morning,movie
